In [ ]:
# hide
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# export
import math
import os
import time
import logging
from pathlib import Path
from typing import Collection

import prefect
import fastparquet
import spell.metrics
from prefect import task, Flow
from prefect.engine.signals import SKIP
from prefect.tasks.shell import ShellTask

from neuralmusic.midi import parse_midi_file

log = logging.getLogger("data.etl")

<IPython.core.display.Javascript object>

In [ ]:
# default_exp data.etl

<IPython.core.display.Javascript object>

# ETL to process MIDI files

> Turning a bunch of MIDI files into parquet data

This ETL takes a tar.gz'd file full of arbitrary midi files and outputs a bunch of parquet files containing neat dataframes with pitch, durations and velocity triplets for each song.

First we need some global variables and reporting tools to log the progress. This ETL may run locally or on Spell.

In [ ]:
# export
total_songs = 0
malformed_songs = 0
valid_songs = 0
valid_notes = 0

started_at = None


def init_stats():
    """
    Resets reporting stats to zero.
    """
    global total_songs
    global valid_songs
    global valid_notes
    global malformed_songs
    global started_at

    total_songs = 0
    malformed_songs = 0
    valid_songs = 0
    valid_notes = 0

    started_at = time.time()


if "SPELL" in os.environ:
    metric = spell.metrics.send_metric

    def metric(_logger, k, v):
        spell.metrics.send_metric(k, v)


else:

    def metric(logger, k, v):
        logger.info(f"[{k}]: {v}")


def report(logger):
    """
    Reports current metrics, either to Spell or to a logger.
    """
    elapsed = time.time() - started_at
    metric(logger, "Total Songs", total_songs)
    metric(logger, "Malformed Songs", malformed_songs)
    metric(logger, "Notes", valid_notes)
    metric(logger, "Total Songs / second", (total_songs / elapsed))
    metric(logger, "Notes / second", (valid_notes / elapsed))

<IPython.core.display.Javascript object>

## Untar'ing the file

The first step is to untar the file containing the MIDI files.

In [ ]:
# export
@task
def untar_cmd(file_path: str, outdir: str) -> str:
    """
    Untars a .tar.gz file onto a directory (will create it if it does not exist).
    """
    if os.path.exists(outdir):
        raise SKIP("Output directory already exists.")
    return f"mkdir -p {outdir} && tar -zxf {file_path} -C {outdir}"


untar = ShellTask(name="untar_task")

<IPython.core.display.Javascript object>

## Partitioning the files in minibatches

Since the tar.gz file may contain a huge amount of MIDI files, we'll partition those files into minibatches that we can process in parallel.

In [ ]:
# export
@task(skip_on_upstream_skip=False)
def partition_files(
    data_path: str, partition_size: int = 100, min_partitions: int = 4
) -> list:
    """
    Partitions the midi files in data_path into chunks.
    """
    midi_files = list(Path(data_path).glob("**/*.mid"))
    n = len(midi_files)
    if (n / partition_size) < min_partitions:
        partition_size = math.ceil(n / min_partitions)
    logger = prefect.context.get("logger")
    logger.info(
        f"Processing {n} MIDI files partitioned into groups of {partition_size}"
    )
    return [
        midi_files[i : i + partition_size]
        for i in range(0, len(midi_files), partition_size)
    ]

<IPython.core.display.Javascript object>

## Processing a minibatch

For each minibatch, we'll go through its MIDI files, parse them, and write them to a separate Parquet file.

In [ ]:
# export
@task
def process_and_write(mini_batch: Collection[str], outdir: str) -> bytes:
    """
    Parses a mini batch of MIDI files and writes the results to a parquet file.
    The filename is determined by `map_index`. Returns the amount of notes it parsed.
    """
    frame_no = prefect.context.get("map_index")
    Path(outdir).mkdir(parents=True, exist_ok=True)
    outfile = f"{outdir}/out_{frame_no}.parq"

    logger = prefect.context.get("logger")

    should_append = False

    global total_songs
    global valid_songs
    global valid_notes
    global malformed_songs

    for file in mini_batch:
        df, processed_notes = parse_midi_file(file)
        if df is not None:
            valid_songs += 1
            valid_notes += processed_notes

            fastparquet.write(outfile, df, compression="SNAPPY", append=should_append)
            del df
            should_append = True
        else:
            malformed_songs += 1
            logger.warning(f"[Minibatch {frame_no}] {file} could not be processed.")

        total_songs += 1
        report(logger)

    return outfile

<IPython.core.display.Javascript object>

## Merging the parquet files

Once we have all the minibatches in separate parquet files, merging them into a single dataset is trivial.

In [ ]:
# export
@task
def combine_parquet_files(files: Collection[str]) -> None:
    """
    Combines N parquet files with the same schema into another one.
    """
    fastparquet.writer.merge(files)

<IPython.core.display.Javascript object>

## Putting everything together

Now we can build the ETL flow!

In [ ]:
# export


def build_etl(cfg):
    """
    Builds the ETL flow.
    """
    tar_gz_path = cfg.tar_gz_path
    outdir = cfg.outdir
    assert tar_gz_path, "Config not found: data.etl.tar_gz_path"
    assert outdir, "Config not found: data.etl.outdir"

    with Flow("Neuralmusic Data ETL") as flow:
        tar_gz_path = Path(cfg.tar_gz_path).resolve()
        assert tar_gz_path.exists(), f"{tar_gz_path} does not exist"
        command = untar_cmd(str(tar_gz_path), "data")
        untarred = untar(command=command)

        mini_batches = partition_files(
            "data", partition_size=cfg.partition_size, upstream_tasks=[untarred]
        )

        partitions = process_and_write.map(mini_batches, outdir=outdir)

        combine_parquet_files(partitions)

    return flow

<IPython.core.display.Javascript object>

In [ ]:
# test
from testing import test_eq, path
from omegaconf import OmegaConf
import fastparquet

tmp_path = "/tmp/neuralmusic_etl"

targz_path = path("data/midi.tar.gz")

dot_list = [f"tar_gz_path={targz_path}", f"outdir={tmp_path}", "partition_size=1"]
etl_cfg = OmegaConf.from_dotlist(dot_list)
flow = build_etl(etl_cfg)

init_stats()

started_at = time.time()
flow.run()

test_eq(3, total_songs)
test_eq(0, malformed_songs)
test_eq(3, valid_songs)
test_eq(1457, valid_notes)

df = fastparquet.ParquetFile(tmp_path, verify=True).to_pandas()
pitches, durations, velocities = df.values[0]

test_eq(["7.11.2", "7", "7"], pitches[0:3])
test_eq([1.75, 0.5, 0.5], durations[0:3])
test_eq([110, 110, 110], velocities[0:3])

[2019-12-08 08:38:06,421] INFO - prefect.FlowRunner | Beginning Flow run for 'Neuralmusic Data ETL'
[2019-12-08 08:38:06,425] INFO - prefect.FlowRunner | Starting flow run.
[2019-12-08 08:38:06,439] INFO - prefect.TaskRunner | Task 'untar_cmd': Starting task run...
[2019-12-08 08:38:06,443] INFO - prefect.TaskRunner | Task 'untar_cmd': finished task run for task with final state: 'Skipped'
[2019-12-08 08:38:06,458] INFO - prefect.TaskRunner | Task 'untar_task': Starting task run...
[2019-12-08 08:38:06,468] INFO - prefect.TaskRunner | Task 'untar_task': finished task run for task with final state: 'Skipped'
[2019-12-08 08:38:06,483] INFO - prefect.TaskRunner | Task 'partition_files': Starting task run...
[2019-12-08 08:38:06,487] INFO - prefect.Task: partition_files | Processing 3 MIDI files partitioned into groups of 1
[2019-12-08 08:38:06,494] INFO - prefect.TaskRunner | Task 'partition_files': finished task run for task with final state: 'Success'
[2019-12-08 08:38:06,507] INFO - pr

<IPython.core.display.Javascript object>